In [ ]:
!sudo pip3 install keras

In [ ]:
import numpy as np
import math
import re
import pandas as pd
import nltk
import string

from bs4 import BeautifulSoup
from google.colab import drive

In [ ]:
try:
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf
from tensorflow.keras import layers
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
import tensorflow_datasets as tfds
from unicodedata import normalize

Montamos drive para recoger los datos

In [ ]:
drive.mount('/content/drive')

# Preparación de los datos





In [ ]:
cols = ["texto", "nota"]
tweet_data = pd.read_csv(
    "/content/drive/MyDrive/Colab Notebooks/Data/alltweets.csv",
    header=None,
    names=cols,
    engine="python",
    encoding="utf8"
)

Mostramos las 5 primeras filas del csv

In [ ]:
tweet_data.head(5)

,texto,nota
0,- Rápida. Utilizamos las últimas tecnologías p...,0
1,‼ ¡Mucho cuidado!\n\n🌞Si te vas a exponer al s...,0
2,‼️ Cuidado. La Guardia Civil advierte sobre mú...,0
3,"‼ El ""gigante"" de la sidra espumosa no logra d...",1
4,"‼ El Gobierno aprueba limitar ""excepcionalment...",1


Los tweets tienen un valor numérico comprendido entre 0 y 10 que indica la relevancia que tiene un tweet.



## Limpieza del texto

In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

Función para limpiar el texto antes de trabajar con él

In [ ]:
def clean_data(tweet):
  tweet = BeautifulSoup(tweet,"lxml").get_text()
  #pasamos a minúsculas
  tweet = tweet.lower()
  #Eliminamos las menciones a usuarios
  tweet = re.sub(r"@[a-z0-9]+", ' ', tweet)
  #Eliminamos los enlaces
  tweet = re.sub(r"https?://[a-z0-9./]", ' ', tweet)
  #Nos quedamos con los caracteres importantes
  tweet = re.sub(r"[^a-zÀ-ÿ']", ' ', tweet)
  #Se quitan los espacios en blanco que se hayan generado de más
  tweet = re.sub(r" +", ' ', tweet) 
  #Se eliminan las tildes  
  tweet = re.sub(
        r"([^n\u0300-\u036f]|n(?!\u0303(?![\u0300-\u036f])))[\u0300-\u036f]+", r"\1", 
        normalize( "NFD", tweet), 0, re.I)    
  #limpiamos ahora con nltk
  notPunctuation = [c for c in tweet if c not in string.punctuation]
  notPunctuation = "".join(notPunctuation)
  words = notPunctuation.split()
  tweet = [word for word in words if word.lower() not in stopwords.words("spanish")]
  tweet = " ".join(tweet)
  return tweet

 Usamos la función para limpiar cada uno de los tweets

In [ ]:
corpus = [clean_data(tweet) for tweet in tweet_data.texto]

In [ ]:
print(corpus[0])

rapida utilizamos ultimas tecnologias cargarla mas rapido viva reservamos espacios destacados informacion vivo contenido multimedia ultima hora directos ordenada articulos estaran claramente agrupados tematicas eventos


In [ ]:
labels = tweet_data.nota.values

##Tokenización

Para cada tweet construimos la lista de palabras que contiene

In [ ]:
tokenizer = tfds.deprecated.text.SubwordTextEncoder.build_from_corpus(
    corpus, target_vocab_size=2**16
)

data = [tokenizer.encode(text) for text in corpus] #para cada tweet una lista de palabras tokenizadas

In [ ]:
data

##Padding

Hacemos que todos los tweets a tratar tengan la misma longitud

In [ ]:
max_len = max([len(text) for text in data])
data = tf.keras.preprocessing.sequence.pad_sequences(data,
                                                     value = 0,
                                                     padding = "post",
                                                     maxlen = max_len)

##División en entrenamiento y test

In [ ]:
tweet_train, tweet_test, train_value, test_value = train_test_split(data, labels, test_size=0.2, random_state=37, stratify = labels)

# Modelo

In [ ]:
class DCNN(tf.keras.Model):
      
      def __init__(self,
                  vocab_size, #tamaño del vocabulario
                  emb_dim=128, #espacio vectorial de 128 para cada palabra
                  nb_filters=50, #numero de filtros
                  FFN_units=512, #neuronas de la capa oculta de la red neuronal
                  nb_classes=2, #2 categorías de clasificación
                  dropout_rate=0.1, #el 10% de las neuronas no transmiten lo aprendido
                  training=False,
                  name="dcnn"):
          super(DCNN, self).__init__(name=name)
          
          #definición de las capas de la red neuronal
          self.embedding = layers.Embedding(vocab_size,
                                            emb_dim)
          self.bigram = layers.Conv1D(filters=nb_filters,
                                      kernel_size=2, #filtra elementos de 2 en dos
                                      padding="valid", #añade ceros cuando haga falta
                                      activation="relu") #función rectificadora lineal unitaria f(x)=max(0,x)
          self.trigram = layers.Conv1D(filters=nb_filters,
                                      kernel_size=3,
                                      padding="valid",
                                      activation="relu")
          self.fourgram = layers.Conv1D(filters=nb_filters,
                                        kernel_size=4,
                                        padding="valid",
                                        activation="relu")
          self.pool = layers.GlobalMaxPool1D() 
          #definimos la red neuronal que se encarga de la clasificación
          self.dense_1 = layers.Dense(units=FFN_units, activation="relu") #la capa oculta
          self.dropout = layers.Dropout(rate=dropout_rate) #la capa oculta
          if nb_classes == 2:
              self.last_dense = layers.Dense(units=1,
                                            activation="sigmoid")
          else: #si no queremos clasificación binaria
              self.last_dense = layers.Dense(units=nb_classes,
                                            activation="softmax")
      
      #input - son los bloques de palabras que se usan para predecir
      #training - booleano que define si estamos o no entrenando
      def call(self, inputs, training):
          x = self.embedding(inputs)
          x_1 = self.bigram(x)
          x_1 = self.pool(x_1)
          x_2 = self.trigram(x)
          x_2 = self.pool(x_2)
          x_3 = self.fourgram(x)
          x_3 = self.pool(x_3)
          
          #preparamos los datos que le vamos a pasar a la red neuronal
          merged = tf.concat([x_1, x_2, x_3], axis=-1)
          merged = self.dense_1(merged)
          merged = self.dropout(merged, training)
          output = self.last_dense(merged)
          
          return output

Variables globales

In [ ]:
VOCABULARY_SIZE = tokenizer.vocab_size
EMB_DIM = 200
NB_FILTERS = 100
FFN_UNITS = 256
NB_CLASSES = 2 
DROPOUT_RATE = 0.2 #tasa de olvido
BATCH_SIZE = 32 #tamaño del bloque
NB_EPOCHS = 4 #numero de veces que pasamos por el dataset

# Entrenamiento

In [ ]:
Dcnn = DCNN(vocab_size=VOCABULARY_SIZE,
            emb_dim=EMB_DIM,
            nb_filters=NB_FILTERS,
            FFN_units=FFN_UNITS,
            nb_classes=NB_CLASSES,
            dropout_rate=DROPOUT_RATE)

In [ ]:
if NB_CLASSES == 2:
  Dcnn.compile(loss="binary_crossentropy",
               optimizer="adam",
               metrics=["accuracy"])
else:
  Dcnn.compile(loss="sparse_categorical_crossentropy",
               optimizer="adam",
               metrics=["sparse_categorical_accuracy"])  

Creamos checkpoints por si nos echa de sesión, para ir guardando la información en Drive

In [ ]:
path = "/content/drive/MyDrive/Colab Notebooks/checkpoints"

checkp = tf.train.Checkpoint(Dcnn=Dcnn)
manager = tf.train.CheckpointManager(checkp, path, max_to_keep=5)

if manager.latest_checkpoint:
  checkp.restore(manager.latest_checkpoint)
  print("Se ha restaurado el último checkpoint")

In [ ]:
Dcnn.fit(tweet_train,
         train_value,
         batch_size=BATCH_SIZE,
         epochs=NB_EPOCHS
         )
manager.save()

#Evaluación

Evaluamos pasando los tweet_test y los test_value para que pueda comparar el resultado que predice la red neuronal con la etiqueta real sobre el conjunto de test

In [ ]:
results = Dcnn.evaluate(tweet_test, test_value, batch_size=BATCH_SIZE)

239/239 [==============================] - 4s 18ms/step - loss: 0.4781 - accuracy: 0.8499


Probamos con un ejemplos

In [ ]:
Dcnn(np.array([tokenizer.encode("Rusia anuncia que reducirá su actividad militar cerca de Kiev y Chernígov")]), training=False).numpy()

In [ ]:
Dcnn(np.array([tokenizer.encode("Detectan retrasos del lenguaje en los niños nacidos durante la pandemia del coronavirus por las mascarillas")]), training=False).numpy()

In [ ]:
Dcnn(np.array([tokenizer.encode("Los colutorios bucales son una herramienta efectiva para reducir la capacidad de infectar de Ómicron")]), training=False).numpy()